In [ ]:
# SQL-7 --> CINDITIONAL FUNCTIONNS IN SQL.
# ---> IF,CASE,COALESCE,NULLIF,IFNULL,

In [2]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
import mysql.connector as m
from tabulate import tabulate
import pandas as pd
mydb = m.connect(host = 'localhost',user = 'root',passwd = 'Loveumom@123')
cursor = mydb.cursor()

In [4]:
cursor.execute('SHOW DATABASES')
print(tabulate(cursor.fetchall(),headers = ['DATABASES'],tablefmt = 'pretty'))

+--------------------+
|     DATABASES      |
+--------------------+
|     cardataset     |
| information_schema |
|       mysql        |
| performance_schema |
|      sidfirst      |
|    sqlpractice1    |
|        sys         |
+--------------------+


In [5]:
cursor.execute('USE sqlpractice1')
cursor.execute('SHOW TABLES')
print(tabulate(cursor.fetchall(),headers = ['TABLES IN sqlpractice1'],tablefmt = 'pretty'))

+------------------------+
| TABLES IN sqlpractice1 |
+------------------------+
|       employees        |
|        student         |
+------------------------+


In [ ]:
cursor.execute("""
CREATE TABLE employees (
    id INT PRIMARY KEY,
    name VARCHAR(50),
    age INT,
    salary DECIMAL(10, 2),
    department VARCHAR(50),
    phone_number VARCHAR(20),
    email VARCHAR(50),
    bonus DECIMAL(10, 2)
);
""") # it will create a new table employees in sqlpractice1 database as above we used USE sqlpractice1 as database

In [8]:
cursor.execute('USE sqlpractice1')
cursor.execute('SHOW TABLES')
print(tabulate(cursor.fetchall(),headers = ['TABLES IN sqlpractice1'],tablefmt = 'pretty'))

+------------------------+
| TABLES IN sqlpractice1 |
+------------------------+
|       employees        |
|        student         |
+------------------------+


In [9]:
cursor.execute("""
INSERT INTO employees (id, name, age, salary, department, phone_number, email, bonus)
VALUES
(1, 'John Doe', 25, 55000, 'Engineering', '1234567890', 'john@example.com', 5000),
(2, 'Jane Smith', 45, 48000, 'Marketing', NULL, 'jane@example.com', NULL),
(3, 'Sam Brown', 62, 70000, 'HR', '0987654321', NULL, 0),
(4, 'Lucy Green', 30, NULL, NULL, NULL, 'lucy@example.com', 2000);
""")

In [10]:
mydb.commit()

In [5]:
cursor.execute("""
SELECT * FROM sqlpractice1.employees
""")

In [6]:
print(tabulate(cursor.fetchall(),headers=[column [0] for column in cursor.description],tablefmt = 'pretty'))

+----+------------+-----+----------+-------------+--------------+------------------+---------+
| id |    name    | age |  salary  | department  | phone_number |      email       |  bonus  |
+----+------------+-----+----------+-------------+--------------+------------------+---------+
| 1  |  John Doe  | 25  | 55000.00 | Engineering |  1234567890  | john@example.com | 5000.00 |
| 2  | Jane Smith | 45  | 48000.00 |  Marketing  |              | jane@example.com |         |
| 3  | Sam Brown  | 62  | 70000.00 |     HR      |  0987654321  |                  |  0.00   |
| 4  | Lucy Green | 30  |          |             |              | lucy@example.com | 2000.00 |
+----+------------+-----+----------+-------------+--------------+------------------+---------+


In [ ]:
# 7.1--> IF FUNCTION :
# Purpose: The IF function is used to return one value if a condition is true and another value if the condition is false.
# Syntax: IF(condition, value_if_true, value_if_false) 

In [27]:
# IF Function # 
#Problem-1 : Write a SQL query to categorize employees based on their salary into TWO categories: "Low" and "High". 
# Consider salaries below 50,000 as "Low" and above 50,000 as "High".

cursor.execute("""
SELECT name,
IF (salary > 50000,'High','Low') AS salary_level
FROM sqlpractice1.employees;
""")

In [28]:
print(tabulate(cursor.fetchall(),headers=[column [0] for column in cursor.description],tablefmt = 'pretty'))

+------------+--------------+
|    name    | salary_level |
+------------+--------------+
|  John Doe  |     High     |
| Jane Smith |     Low      |
| Sam Brown  |     High     |
| Lucy Green |     Low      |
+------------+--------------+


In [29]:
# Issue in above result : Lucy Green has empty salary value still in result it shows "Low".
# now how to get "No salary Data" for salaries that has NULL value like lucy green, using IF function.
cursor.execute("""
SELECT name,salary,
       IF(salary IS NULL, 'No Salary Data', IF(salary > 50000, 'High', 'Low')) AS salary_level
FROM sqlpractice1.employees;
""")

In [30]:
print(tabulate(cursor.fetchall(),headers=[column [0] for column in cursor.description],tablefmt = 'pretty'))

+------------+----------+----------------+
|    name    |  salary  |  salary_level  |
+------------+----------+----------------+
|  John Doe  | 55000.00 |      High      |
| Jane Smith | 48000.00 |      Low       |
| Sam Brown  | 70000.00 |      High      |
| Lucy Green |          | No Salary Data |
+------------+----------+----------------+


In [31]:
# 7.2--> CASE FUNCTION :
# Purpose: The CASE function allows you to perform complex conditional queries. It returns values based on multiple conditions.
# Syntax:
"""
CASE 
    WHEN condition1 THEN result1
    WHEN condition2 THEN result2
    ...
    ELSE result
END
"""

'\nCASE \n    WHEN condition1 THEN result1\n    WHEN condition2 THEN result2\n    ...\n    ELSE result\nEND\n'

In [32]:
# CASE FUNCTION
# problem-1: dividing employees into three categories according to their age_group. age >= 18 < 60 as adult,
# age < 18 as minor and otherwise senior

cursor.execute("""
SELECT name,age,
        CASE
            WHEN age < 18 THEN 'Minor'
            WHEN age >= 18 AND age < 60 THEN 'Adult'
            ELSE 'Senior'
        END AS Age_Group
FROM sqlpractice1.employees;
""")

In [33]:
print(tabulate(cursor.fetchall(),headers=[column [0] for column in cursor.description],tablefmt = 'pretty'))

+------------+-----+-----------+
|    name    | age | Age_Group |
+------------+-----+-----------+
|  John Doe  | 25  |   Adult   |
| Jane Smith | 45  |   Adult   |
| Sam Brown  | 62  |  Senior   |
| Lucy Green | 30  |   Adult   |
+------------+-----+-----------+


In [21]:
# 7.3 COALESCE FUNCTION : 
# Purpose: The COALESCE function returns the first non-null value in the list of expressions.
# Syntax: COALESCE(expression1, expression2, ..., expression_n)
cursor.execute("""
SELECT *, COALESCE(phone_number, email, 'No Contact Info') AS contact_info
FROM sqlpractice1.employees;
""")
# This returns the first non-null contact information available (phone number, email, or 'No Contact Info').

In [22]:
print(tabulate(cursor.fetchall(),headers=[column [0] for column in cursor.description],tablefmt = 'pretty'))

+----+------------+-----+----------+-------------+--------------+------------------+---------+------------------+
| id |    name    | age |  salary  | department  | phone_number |      email       |  bonus  |   contact_info   |
+----+------------+-----+----------+-------------+--------------+------------------+---------+------------------+
| 1  |  John Doe  | 25  | 55000.00 | Engineering |  1234567890  | john@example.com | 5000.00 |    1234567890    |
| 2  | Jane Smith | 45  | 48000.00 |  Marketing  |              | jane@example.com |         | jane@example.com |
| 3  | Sam Brown  | 62  | 70000.00 |     HR      |  0987654321  |                  |  0.00   |    0987654321    |
| 4  | Lucy Green | 30  |          |             |              | lucy@example.com | 2000.00 | lucy@example.com |
+----+------------+-----+----------+-------------+--------------+------------------+---------+------------------+


In [19]:
# 7.4 NULLIF :
# Purpose: The NULLIF function returns NULL if two expressions are equal; otherwise, it returns the first expression.
# Syntax: NULLIF(expression1, expression2)
cursor.execute("""
SELECT *, NULLIF(bonus, 0) AS final_bonus
FROM sqlpractice1.employees;
""")
# it returns NULL if two expression are equal,otherwise it returns first value.
# (5000,0)--->5000
# (NULL,0)--->NULL
# (0,0) ----->NULL
# (2000,0)--->2000

In [20]:
print(tabulate(cursor.fetchall(),headers=[column [0] for column in cursor.description],tablefmt = 'pretty'))

+----+------------+-----+----------+-------------+--------------+------------------+---------+-------------+
| id |    name    | age |  salary  | department  | phone_number |      email       |  bonus  | final_bonus |
+----+------------+-----+----------+-------------+--------------+------------------+---------+-------------+
| 1  |  John Doe  | 25  | 55000.00 | Engineering |  1234567890  | john@example.com | 5000.00 |   5000.00   |
| 2  | Jane Smith | 45  | 48000.00 |  Marketing  |              | jane@example.com |         |             |
| 3  | Sam Brown  | 62  | 70000.00 |     HR      |  0987654321  |                  |  0.00   |             |
| 4  | Lucy Green | 30  |          |             |              | lucy@example.com | 2000.00 |   2000.00   |
+----+------------+-----+----------+-------------+--------------+------------------+---------+-------------+


In [25]:
# 7.5 IFNULL FUNCTION:
# Purpose: The IFNULL function returns the first expression if it's not null; otherwise, it returns the second expression.
# Syntax: IFNULL(expression1, expression2)

cursor.execute("""
SELECT *,IFNULL(department, 'No Department') AS department_info
FROM sqlpractice1.employees;
""")
# This will return 'No Department' if the department is NULL.

In [26]:
print(tabulate(cursor.fetchall(),headers=[column [0] for column in cursor.description],tablefmt = 'pretty'))

+----+------------+-----+----------+-------------+--------------+------------------+---------+-----------------+
| id |    name    | age |  salary  | department  | phone_number |      email       |  bonus  | department_info |
+----+------------+-----+----------+-------------+--------------+------------------+---------+-----------------+
| 1  |  John Doe  | 25  | 55000.00 | Engineering |  1234567890  | john@example.com | 5000.00 |   Engineering   |
| 2  | Jane Smith | 45  | 48000.00 |  Marketing  |              | jane@example.com |         |    Marketing    |
| 3  | Sam Brown  | 62  | 70000.00 |     HR      |  0987654321  |                  |  0.00   |       HR        |
| 4  | Lucy Green | 30  |          |             |              | lucy@example.com | 2000.00 |  No Department  |
+----+------------+-----+----------+-------------+--------------+------------------+---------+-----------------+


In [40]:
# problem : NULLIF 
# Write a SQL query to calculate the bonus percentage for each employee. If the bonus is 0, the percentage should be NULL.
cursor.execute("""
SELECT salary,bonus, 
       (bonus / NULLIF(salary, 0)) * 100 AS bonus_percentage
FROM sqlpractice1.employees;
""")

In [41]:
print(tabulate(cursor.fetchall(),headers=[column [0] for column in cursor.description],tablefmt = 'pretty'))

+----------+---------+------------------+
|  salary  |  bonus  | bonus_percentage |
+----------+---------+------------------+
| 55000.00 | 5000.00 |     9.090909     |
| 48000.00 |         |                  |
| 70000.00 |  0.00   |     0.000000     |
|          | 2000.00 |                  |
+----------+---------+------------------+


In [44]:
# another approach to solve above problem. and this is more accurate as we needed.
cursor.execute("""
SELECT name,salary,bonus, 
       CASE 
           WHEN salary IS NULL OR bonus = 0 THEN NULL
           ELSE (bonus / salary) * 100
       END AS bonus_percentage
FROM sqlpractice1.employees;
""")

In [45]:
print(tabulate(cursor.fetchall(),headers=[column [0] for column in cursor.description],tablefmt = 'pretty'))

+------------+----------+---------+------------------+
|    name    |  salary  |  bonus  | bonus_percentage |
+------------+----------+---------+------------------+
|  John Doe  | 55000.00 | 5000.00 |     9.090909     |
| Jane Smith | 48000.00 |         |                  |
| Sam Brown  | 70000.00 |  0.00   |                  |
| Lucy Green |          | 2000.00 |                  |
+------------+----------+---------+------------------+


In [49]:
# one another solution for above using IF function
cursor.execute("""
SELECT name,salary,bonus, IF(salary IS NULL OR bonus=0,NULL,(bonus/salary)*100) AS bonus_percentage
FROM sqlpractice1.employees
""")

In [50]:
print(tabulate(cursor.fetchall(),headers=[column [0] for column in cursor.description],tablefmt = 'pretty'))

+------------+----------+---------+------------------+
|    name    |  salary  |  bonus  | bonus_percentage |
+------------+----------+---------+------------------+
|  John Doe  | 55000.00 | 5000.00 |     9.090909     |
| Jane Smith | 48000.00 |         |                  |
| Sam Brown  | 70000.00 |  0.00   |                  |
| Lucy Green |          | 2000.00 |                  |
+------------+----------+---------+------------------+


In [58]:
# problem : Combine Multiple Conditions:
# Question: Write a SQL query to find employees who either do not have a phone number or have a salary less than 50,000.

cursor.execute("""
SELECT * FROM sqlpractice1.employees 
WHERE phone_number IS NULL OR salary < 50000;
""")

In [59]:
print(tabulate(cursor.fetchall(),headers=[column [0] for column in cursor.description],tablefmt = 'pretty'))

+----+------------+-----+----------+------------+--------------+------------------+---------+
| id |    name    | age |  salary  | department | phone_number |      email       |  bonus  |
+----+------------+-----+----------+------------+--------------+------------------+---------+
| 2  | Jane Smith | 45  | 48000.00 | Marketing  |              | jane@example.com |         |
| 4  | Lucy Green | 30  |          |            |              | lucy@example.com | 2000.00 |
+----+------------+-----+----------+------------+--------------+------------------+---------+


In [1]:
# ------------------------> CHALLENGES UDEMY  : CONDITIONAL FUNCTIONS <-------------------------------------------

In [ ]:
# CHALLENGE :1 
# StackOverflow publishes the results of the survey about popular databases amongst developers every year.  
# I have used the information to create a table called popular_databases with 4 columns 
# database_name, vote_percentage_2023, vote_percentage_2022, vote_percentage_2021.

# Please write a SELECT statement to query the table popular_databases and output the following columns

# 1. database_name

# 2. vote_year - This column is populated with the latest year for which the data is available for. 
# If the data is available for none of the years, please output the value 'N/A'

# 3.vote_percentage - This column should have the vote percentage for the latest year for which the data is available. 
# If the data is available for none of the years, please output the value 'N/A'.

In [8]:
cursor.execute("""
CREATE TABLE sqlpractice1.popular_databases(
database_name VARCHAR(50),
vote_percent_23 INT,
vote_percent_22 INT,
vote_percent_21 INT)
""")

In [9]:
cursor.execute("""
ALTER TABLE sqlpractice1.popular_databases
MODIFY COLUMN vote_percent_23 DECIMAL(4,2),
MODIFY COLUMN vote_percent_22 DECIMAL(4,2),
MODIFY COLUMN vote_percent_21 DECIMAL(4,2);
""")

In [19]:
cursor.execute("""
DESCRIBE sqlpractice1.popular_databases
""")

In [20]:
cursor.fetchall()

[('database_name', 'varchar(50)', 'YES', '', None, ''),
 ('vote_percent_23', 'decimal(4,2)', 'YES', '', None, ''),
 ('vote_percent_22', 'decimal(4,2)', 'YES', '', None, ''),
 ('vote_percent_21', 'decimal(4,2)', 'YES', '', None, '')]

In [15]:
cursor.execute("""
INSERT INTO sqlpractice1.popular_databases(database_name,vote_percent_23,vote_percent_22,vote_percent_21)
VALUES 
('PostgreSQL',45.55,NULL,40.42),
('MYSQL',41.09,46.85,NULL),
('SQLite',NULL,32.01,32.18),
('MongoDB',NULL,NULL,27.7),
('MYSQLserver',NULL,NULL,NULL);
""")

In [16]:
mydb.commit()

In [17]:
cursor.execute("""
SELECT * FROM sqlpractice1.popular_databases
""")

In [18]:
print(tabulate(cursor.fetchall(),headers=[column [0] for column in cursor.description],tablefmt = 'pretty'))

+---------------+-----------------+-----------------+-----------------+
| database_name | vote_percent_23 | vote_percent_22 | vote_percent_21 |
+---------------+-----------------+-----------------+-----------------+
|  PostgreSQL   |      45.55      |                 |      40.42      |
|     MYSQL     |      41.09      |      46.85      |                 |
|    SQLite     |                 |      32.01      |      32.18      |
|    MongoDB    |                 |                 |      27.70      |
|  MYSQLserver  |                 |                 |                 |
+---------------+-----------------+-----------------+-----------------+


In [30]:
# solution :1.1
cursor.execute("""
SELECT 
    database_name,
    CASE 
        WHEN vote_percent_23 IS NOT NULL THEN '2023'
        WHEN vote_percent_22 IS NOT NULL THEN '2022'
        WHEN vote_percent_21 IS NOT NULL THEN '2021'
        ELSE 'N/A'
    END AS vote_year,
    CASE 
        WHEN vote_percent_23 IS NOT NULL THEN vote_percent_23
        WHEN vote_percent_22 IS NOT NULL THEN vote_percent_22
        WHEN vote_percent_21 IS NOT NULL THEN vote_percent_21
        ELSE 'N/A'
    END AS vote_percentage
FROM sqlpractice1.popular_databases;
""")
# Explanation:
# 1.vote_year: This column checks for the availability of data in descending order (starting from 2023) and outputs 
# the corresponding year if data is available; otherwise, it outputs 'N/A'.
# 2.vote_percentage: This column retrieves the vote percentage from the latest year available. 
# If no data is available for any year, it returns 'N/A'.

In [31]:
print(tabulate(cursor.fetchall(),headers=[column [0] for column in cursor.description],tablefmt = 'pretty'))

+---------------+-----------+-----------------+
| database_name | vote_year | vote_percentage |
+---------------+-----------+-----------------+
|  PostgreSQL   |   2023    |      45.55      |
|     MYSQL     |   2023    |      41.09      |
|    SQLite     |   2022    |      32.01      |
|    MongoDB    |   2021    |      27.70      |
|  MYSQLserver  |    N/A    |       N/A       |
+---------------+-----------+-----------------+


In [28]:
# solution : 1.2 using COLEASCE function
cursor.execute("""
SELECT 
    database_name,
    COALESCE(
        CASE WHEN vote_percent_23 IS NOT NULL THEN '2023' END,
        CASE WHEN vote_percent_22 IS NOT NULL THEN '2022' END,
        CASE WHEN vote_percent_21 IS NOT NULL THEN '2021' END,
        'N/A'
    ) AS vote_year,
    COALESCE(vote_percent_23, vote_percent_22, vote_percent_21, 'N/A') AS vote_percentage
FROM sqlpractice1.popular_databases;
""")
# COALESCE works by returning the first non-NULL value from a list of expressions. In this case:
# For vote_year, we use CASE statements inside COALESCE to return the latest available year where the vote percentage is not NULL. 
# If no year has data, it returns 'N/A'.
# For vote_percentage, it picks the first non-NULL vote percentage from 2023, 2022, or 2021. 
# If none are available, it returns 'N/A'.

In [29]:
print(tabulate(cursor.fetchall(),headers=[column [0] for column in cursor.description],tablefmt = 'pretty'))

+---------------+-----------+-----------------+
| database_name | vote_year | vote_percentage |
+---------------+-----------+-----------------+
|  PostgreSQL   |   2023    |      45.55      |
|     MYSQL     |   2023    |      41.09      |
|    SQLite     |   2022    |      32.01      |
|    MongoDB    |   2021    |      27.70      |
|  MYSQLserver  |    N/A    |       N/A       |
+---------------+-----------+-----------------+


In [38]:
# solution : 1.3 using IFNULL
cursor.execute("""
SELECT 
    *,
    IFNULL(
        IF(vote_percent_23 IS NOT NULL, '2023', NULL),
        IF(vote_percent_22 IS NOT NULL, '2022', IF(vote_percent_21 IS NOT NULL, '2021', 'N/A'))
    ) AS vote_year,
    IFNULL(vote_percent_23, 
    IFNULL(vote_percent_22, IFNULL(vote_percent_21, 'N/A'))) AS vote_percentage
FROM popular_databases;
""")
# Explanation:
# IFNULL checks each condition and if a NULL is encountered, it moves to the next value.
# The inner IF conditions are used to check for the latest year where the vote percentage is not NULL, 
# and then IFNULL returns the first non-NULL value.

In [39]:
print(tabulate(cursor.fetchall(),headers=[column [0] for column in cursor.description],tablefmt = 'pretty'))

+---------------+-----------------+-----------------+-----------------+-----------+-----------------+
| database_name | vote_percent_23 | vote_percent_22 | vote_percent_21 | vote_year | vote_percentage |
+---------------+-----------------+-----------------+-----------------+-----------+-----------------+
|  PostgreSQL   |      45.55      |                 |      40.42      |   2023    |      45.55      |
|     MYSQL     |      41.09      |      46.85      |                 |   2023    |      41.09      |
|    SQLite     |                 |      32.01      |      32.18      |   2022    |      32.01      |
|    MongoDB    |                 |                 |      27.70      |   2021    |      27.70      |
|  MYSQLserver  |                 |                 |                 |    N/A    |       N/A       |
+---------------+-----------------+-----------------+-----------------+-----------+-----------------+


In [ ]:
# challenge : 2 
# StackOverflow publishes the results of the survey about popular programming languages amongst developers every year.  
# I have used the information to create a table called popular_languages with 
# 4 columns language, vote_percentage_2023, vote_percentage_2022, vote_percentage_2021
# Please write a SELECT statement to query the table popular_languages and output the following columns

# 1.language

# 2.vote_year -  This column is populated with the latest year for which the data is available for. 
# If the data is available for none of the years, please output the value 'N/A'

# 3.vote_percentage - This column should have the vote percentage for the latest year for which the data is available. 
# If the data is available for none of the years, please output the value 'N/A'

In [40]:
cursor.execute("""
CREATE TABLE sqlpractice1.popular_languages (
    language VARCHAR(50),
    vote_percentage_2023 DECIMAL(5, 2),
    vote_percentage_2022 DECIMAL(5, 2),
    vote_percentage_2021 DECIMAL(5, 2)
);
""")

In [41]:
cursor.execute("""
INSERT INTO popular_languages (language, vote_percentage_2023, vote_percentage_2022, vote_percentage_2021)
VALUES
('JavaScript', 63.61, 0, 64.96),
('HTML', 52.97, 55.08, 0),
('Python', 0, 48.07, 48.24),
('SQL', 48.66, 0, 47.08),
('TypeScript', 0, 0, 0);
""")

In [42]:
mydb.commit()

In [45]:
cursor.execute("""
SELECT * FROM sqlpractice1.popular_languages
""")

In [46]:
print(tabulate(cursor.fetchall(),headers=[column [0] for column in cursor.description],tablefmt = 'pretty'))

+------------+----------------------+----------------------+----------------------+
|  language  | vote_percentage_2023 | vote_percentage_2022 | vote_percentage_2021 |
+------------+----------------------+----------------------+----------------------+
| JavaScript |        63.61         |         0.00         |        64.96         |
|    HTML    |        52.97         |        55.08         |         0.00         |
|   Python   |         0.00         |        48.07         |        48.24         |
|    SQL     |        48.66         |         0.00         |        47.08         |
| TypeScript |         0.00         |         0.00         |         0.00         |
+------------+----------------------+----------------------+----------------------+


In [49]:
# Solution : 2.1 using case when

cursor.execute("""
SELECT 
    language,
    CASE
        WHEN vote_percentage_2023 > 0 THEN '2023'
        WHEN vote_percentage_2022 > 0 THEN '2022'
        WHEN vote_percentage_2021 > 0 THEN '2021'
        ELSE 'N/A'
    END AS vote_year,
    CASE
        WHEN vote_percentage_2023 > 0 THEN vote_percentage_2023
        WHEN vote_percentage_2022 > 0 THEN vote_percentage_2022
        WHEN vote_percentage_2021 > 0 THEN vote_percentage_2021
        ELSE 'N/A'
    END AS vote_percentage
FROM popular_languages;
""")

#The CASE statement for vote_year checks the columns in reverse order (starting from the most recent year, 2023). 
#It selects the latest year where the vote percentage is greater than 0.
#The second CASE statement for vote_percentage similarly selects the vote percentage for the latest year where data is available.
#If no vote percentage data is available for a language, both vote_year and vote_percentage will be set to 'N/A'

In [50]:
print(tabulate(cursor.fetchall(),headers=[column [0] for column in cursor.description],tablefmt = 'pretty'))

+------------+-----------+-----------------+
|  language  | vote_year | vote_percentage |
+------------+-----------+-----------------+
| JavaScript |   2023    |      63.61      |
|    HTML    |   2023    |      52.97      |
|   Python   |   2022    |      48.07      |
|    SQL     |   2023    |      48.66      |
| TypeScript |    N/A    |       N/A       |
+------------+-----------+-----------------+


In [53]:
# solution:2.2
cursor.execute("""
SELECT 
    language,
    COALESCE(
        NULLIF(vote_percentage_2023, 0), 
        NULLIF(vote_percentage_2022, 0), 
        NULLIF(vote_percentage_2021, 0),
        'N/A'
    ) AS vote_percentage,
    CASE 
        WHEN vote_percentage_2023 > 0 THEN '2023'
        WHEN vote_percentage_2022 > 0 THEN '2022'
        WHEN vote_percentage_2021 > 0 THEN '2021'
        ELSE 'N/A'
    END AS vote_year
FROM popular_languages;
""")

# The COALESCE function is used to find the first non-zero vote percentage. 
# We use NULLIF(column, 0) to convert 0s to NULL since COALESCE skips NULL values.
# The CASE statement for vote_year remains unchanged but still selects the latest year based on the same logic.

In [54]:
print(tabulate(cursor.fetchall(),headers=[column [0] for column in cursor.description],tablefmt = 'pretty'))

+------------+-----------------+-----------+
|  language  | vote_percentage | vote_year |
+------------+-----------------+-----------+
| JavaScript |      63.61      |   2023    |
|    HTML    |      52.97      |   2023    |
|   Python   |      48.07      |   2022    |
|    SQL     |      48.66      |   2023    |
| TypeScript |       N/A       |    N/A    |
+------------+-----------------+-----------+
